# PS-matchning (1:1 Nearest Neighbor + caliper på logit(PS))
Denna notebook implementerar propensity score matching mellan CCI>5 (treated) och CCI≤5 (control) med 1:1 nearest neighbor utan replacement och caliper på logit(PS).


In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))
print("PROJECT_ROOT =", PROJECT_ROOT)

from src.connect_db import run_query, close_tunnel

### Läs in kohort och kovariater från SQL

In [ ]:
# --- Kohort/basdata ---
sql_path = PROJECT_ROOT / "sql" / "stroke_cci_ps_cohort.sql"
with open(sql_path, encoding="utf-8") as f:
    cohort_sql = f.read()
df_base = run_query(cohort_sql)

print("df_base kolumner:", df_base.columns.tolist())
print("df_base rader:", len(df_base))

# --- Kovariater (first 24h) ---
sql_path = PROJECT_ROOT / "sql" / "stroke_covariates_first24h.sql"
with open(sql_path, encoding="utf-8") as f:
    cov_sql = f.read()
df_cov = run_query(cov_sql)

print("df_cov kolumner:", df_cov.columns.tolist())
print("df_cov rader:", len(df_cov))


## Merge och definiera variabler
Vi sammanfogar basdata och 24h-kovariater med en stabil nyckel och skapar en binär könsvariabel.

### Merge + kolumnval

In [ ]:
candidate_keys = ["stay_id", "subject_id", "hadm_id"]
keys_in_both = [k for k in candidate_keys if k in df_base.columns and k in df_cov.columns]
print("Keys i båda:", keys_in_both)

if "stay_id" in keys_in_both:
    merge_keys = ["stay_id"]
elif all(k in keys_in_both for k in ["subject_id", "hadm_id"]):
    merge_keys = ["subject_id", "hadm_id"]
elif "hadm_id" in keys_in_both:
    merge_keys = ["hadm_id"]
else:
    raise ValueError(f"Hittar inga rimliga merge-keys. Keys i båda: {keys_in_both}")

print("Merge keys:", merge_keys)

df = df_base.merge(df_cov, on=merge_keys, how="left")
print("Efter merge:", df.shape)

### Variabeldefinitioner (Välj kolumner robust) 

In [ ]:
# Treatment
TREAT_COL = "high_cci"  # 1 = CCI>5, 0 = CCI≤5

# Ålder/kön kan heta anchor_age eller anchor_age_x beroende på merge
AGE_COL = "anchor_age" if "anchor_age" in df.columns else ("anchor_age_x" if "anchor_age_x" in df.columns else None)
GENDER_COL = "gender" if "gender" in df.columns else ("gender_x" if "gender_x" in df.columns else None)

if AGE_COL is None or GENDER_COL is None:
    raise ValueError(f"Hittar inte ålder/kön. AGE_COL={AGE_COL}, GENDER_COL={GENDER_COL}")

# Labs
GLU_COL = "glucose_first"
NA_COL  = "sodium_first"
CR_COL  = "creatinine_first"
BUN_COL = "bun_first"

# Outcomes
OUTCOMES = [c for c in ["died_30d", "died_90d", "died_1y"] if c in df.columns]

need_exist = [TREAT_COL, AGE_COL, GENDER_COL, GLU_COL, NA_COL, CR_COL, BUN_COL]
missing = [c for c in need_exist if c not in df.columns]
if missing:
    raise ValueError(f"Saknar kolumner i df: {missing}")

print("AGE_COL:", AGE_COL, "GENDER_COL:", GENDER_COL)
print("OUTCOMES:", OUTCOMES)


### Bygg df_model (komplettfall) + PS

In [ ]:
# Skapa binär kön
df["is_male"] = (
    df[GENDER_COL].astype(str).str.strip().str.upper()
      .isin(["M", "MALE", "MAN"])
      .astype(int)
)
SEX_COL = "is_male"

cov_need = [TREAT_COL, AGE_COL, SEX_COL, GLU_COL, NA_COL, CR_COL, BUN_COL]
df_model = df.dropna(subset=cov_need).copy()

df_model["treat"] = df_model[TREAT_COL].astype(int)
# Säkerställ att treat är binär (0/1)
u = set(df_model["treat"].dropna().unique())
if not u.issubset({0, 1}):
    raise ValueError(f"treat är inte binär 0/1. Unika värden: {sorted(u)}") 
# log1p-transform för BUN och kreatinin (för PS)
df_model["log_creatinine"] = np.log1p(df_model[CR_COL].astype(float))
df_model["log_bun"]        = np.log1p(df_model[BUN_COL].astype(float))

covs = [AGE_COL, SEX_COL, GLU_COL, NA_COL, "log_creatinine", "log_bun"]

X = df_model[covs].astype(float).values
t = df_model["treat"].values

psm = LogisticRegression(max_iter=5000, solver="lbfgs")
psm.fit(X, t)

ps = np.clip(psm.predict_proba(X)[:, 1], 1e-6, 1-1e-6)
df_model["ps"] = ps
df_model["logit_ps"] = np.log(ps/(1-ps))

print("df_model:", df_model.shape)
print("treat counts:\n", df_model["treat"].value_counts())
print(df_model.groupby("treat")["ps"].describe()[["min","max","mean","std"]])


### 1:1 Nearest Neighbor-matchning med caliper
Matchning görs utan replacement med caliper = 0.2 × SD(logit(PS)).


In [ ]:
caliper = 0.2 * df_model["logit_ps"].std(ddof=1)
print("Caliper:", caliper)

T = df_model[df_model["treat"] == 1].copy().sort_values("logit_ps", ascending=False)
C = df_model[df_model["treat"] == 0].copy()

nn = NearestNeighbors(n_neighbors=min(50, len(C)))
nn.fit(C[["logit_ps"]].values)

used_controls = set()
pairs = []
unmatched = 0

for tidx, tr in T.iterrows():
    dists, idxs = nn.kneighbors([[tr["logit_ps"]]], return_distance=True)
    found = False
    for d, j in zip(dists[0], idxs[0]):
        cidx = C.index[int(j)]
        if cidx in used_controls:
            continue
        if float(d) <= caliper:
            used_controls.add(cidx)
            pairs.append((tidx, cidx, float(d)))
            found = True
            break
    if not found:
        unmatched += 1

n_pairs = len(pairs)
match_rate = n_pairs / len(T) if len(T) else np.nan

print("Matchade par:", n_pairs)
print("Unmatched treated:", unmatched)
print("Matchningsgrad:", round(match_rate, 4))

if n_pairs == 0:
    raise ValueError("Inga matchade par skapades (kontrollera overlap/caliper).")

mt = [p[0] for p in pairs]
mc = [p[1] for p in pairs]

df_t = df_model.loc[mt].copy()
df_c = df_model.loc[mc].copy()
df_t["match_group"] = range(n_pairs)
df_c["match_group"] = range(n_pairs)

df_matched = (
    pd.concat([df_t, df_c], axis=0)
      .sort_values(["match_group", "treat"])
      .reset_index(drop=True)
)

print("df_matched:", df_matched.shape)
print(df_matched["treat"].value_counts())


### Kovariatbalans före/efter (SMD)
Vi beräknar Standardized Mean Difference för samma kovariater som ingick i PS-modellen.


In [ ]:
def smd_signed(df_in, col, treat_col="treat"):
    a = df_in[df_in[treat_col] == 1][col].astype(float)
    b = df_in[df_in[treat_col] == 0][col].astype(float)
    sp = np.sqrt((a.var(ddof=1) + b.var(ddof=1)) / 2)
    if sp == 0 or np.isnan(sp):
        return 0.0
    return (a.mean() - b.mean()) / sp

smd_tbl = pd.DataFrame({
    "covariate": covs,
    "SMD_pre":  [smd_signed(df_model, c) for c in covs],
    "SMD_post": [smd_signed(df_matched, c) for c in covs],
})
smd_tbl["|SMD_pre|"]  = smd_tbl["SMD_pre"].abs()
smd_tbl["|SMD_post|"] = smd_tbl["SMD_post"].abs()

display(smd_tbl.sort_values("|SMD_pre|", ascending=False))
print("Max |SMD| före :", smd_tbl["|SMD_pre|"].max())
print("Max |SMD| efter:", smd_tbl["|SMD_post|"].max())

# Love-plot enkel
plot_df = smd_tbl.sort_values("|SMD_pre|", ascending=True).reset_index(drop=True)
y = np.arange(len(plot_df))

plt.figure()
plt.scatter(plot_df["SMD_pre"],  y, label="Före")
plt.scatter(plot_df["SMD_post"], y, label="Efter")
plt.axvline(0)
plt.axvline(0.1, linestyle="--")
plt.axvline(-0.1, linestyle="--")
plt.yticks(y, plot_df["covariate"])
plt.xlabel("SMD")
plt.ylabel("Kovariat")
plt.title("Kovariatbalans före och efter PS-matchning")
plt.legend()
plt.grid(True, axis="x")
plt.show()


### Utfallstabeller
Vi redovisar antal döda per grupp i den matchade populationen.


### Risk före/efter (rapporttabell)
Vi sammanställer N, döda och risk% före och efter matchning för varje utfall.


In [ ]:
def deaths_table_pre_post(df_pre, df_post, outcomes):
    rows = []
    for oc in outcomes:
        dpre = df_pre.dropna(subset=[oc]).copy()
        dpost = df_post.dropna(subset=[oc]).copy()

        for dset, tag in [(dpre, "Före"), (dpost, "Efter")]:
            n0 = int((dset["treat"]==0).sum())
            n1 = int((dset["treat"]==1).sum())
            dead0 = int(dset.loc[dset["treat"]==0, oc].sum())
            dead1 = int(dset.loc[dset["treat"]==1, oc].sum())
            r0 = 100*dead0/n0 if n0 else np.nan
            r1 = 100*dead1/n1 if n1 else np.nan

            rows.append({
                "Outcome": oc,
                "Status": tag,
                "N (CCI≤5)": n0,
                "Döda (CCI≤5)": dead0,
                "Risk% (CCI≤5)": r0,
                "N (CCI>5)": n1,
                "Döda (CCI>5)": dead1,
                "Risk% (CCI>5)": r1,
            })

    out = pd.DataFrame(rows)
    out[["Risk% (CCI≤5)", "Risk% (CCI>5)"]] = out[["Risk% (CCI≤5)", "Risk% (CCI>5)"]].round(2)
    return out

df_deaths = deaths_table_pre_post(df_model, df_matched, OUTCOMES)
display(df_deaths)
